## Installation

In [1]:
!python -m pip install langchain==0.1.0 openai==1.7.2 langchain-openai==0.0.2 langchain-community==0.0.12 langchainhub==0.1.14

  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
INFO: pip is looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.9 MB/s eta 0:00:00
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
   ━━━━━━━━━━━━━━━━━

### Load In Envrironment Variables

In [2]:
from dotenv import load_dotenv
import os


In [3]:
%load_ext dotenv
%dotenv

In [4]:
from langchain_openai import ChatOpenAI

In [5]:
llm = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"),
                 model="gpt-4o-mini"
                )

In [6]:
llm.invoke("Hello there!")

AIMessage(content='Hello! How can I assist you today?')

### Working With Message Types

In [7]:
from langchain_core.messages import HumanMessage, SystemMessage

In [8]:
messages = [
    SystemMessage("You are an AI assistant design to tell funny jokes. Do not answer any question that is not a joke."),
    HumanMessage("Tell me a joke."),
]

In [9]:
response = llm.invoke(messages)
print(response.content)

Why did the scarecrow win an award? 

Because he was outstanding in his field!


### Prompt Templates

In [10]:
username = "Prince"
txt = f"Hello {username}"

print(txt)

Hello Prince


In [11]:
from langchain_core.prompts import PromptTemplate

In [12]:
prompt_template = PromptTemplate.from_template(
    "Tell me historical fact about the {event} in {location}.",
)


In [13]:
prompt_template.format(event="World War II", location="Europe")

'Tell me historical fact about the World War II in Europe.'

In [14]:
from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

In [15]:
system_message_str = """You are a helpful AI assistant. Given the 
following context, answer the question. If the answer can not be 
found in the context, simply say you DO NOT KNOW.
Context: {context}
"""

In [16]:
system_message_promt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"],
        template=system_message_str,
    )
)

In [17]:
human_message_str = "Can you provide details on:  {question}"

In [18]:
human_message_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["question"],
        template=human_message_str,
    )
)

In [19]:
messages = [system_message_promt, human_message_prompt]

In [20]:
chatbot_prompt_template = ChatPromptTemplate(
    messages=messages,
    input_variables=["context", "question"],
)

In [21]:
question = "What is the capital of France?"
context = "France is a country in Europe. It's capital is Paris."

In [22]:
chatbot_prompt_template.format(context=context, question=question)

"System: You are a helpful AI assistant. Given the \nfollowing context, answer the question. If the answer can not be \nfound in the context, simply say you DO NOT KNOW.\nContext: France is a country in Europe. It's capital is Paris.\n\nHuman: Can you provide details on:  What is the capital of France?"

In [23]:
response = llm.invoke(
    chatbot_prompt_template.format(context=context, question="What is the capital of Kenya?")
)

In [24]:
response

AIMessage(content='DO NOT KNOW.')

In [25]:
response.content

'DO NOT KNOW.'

### LangChain Expression Language

In [26]:
from langchain_core.output_parsers import StrOutputParser

In [27]:
chain = chatbot_prompt_template | llm | StrOutputParser()

In [28]:
chain.invoke({
    "context": context,
    "question": question
})

'The capital of France is Paris.'

In [29]:
chain.invoke({
    "context": context,
    "question": "What is the capital of Kenya?"
})

'I DO NOT KNOW.'

### LangChain Agents

In [30]:
from langchain import hub
from langchain.agents import (
    AgentExecutor,
    create_openai_tools_agent
)
from langchain.tools import tool
from langchain_core.tools import ToolException
from typing import Literal

In [31]:
operator_type = Literal["add", "multiply", "substraction", "division"]

In [32]:
@tool("Calculator-tool", return_direct=False)
def calculator(operator: operator_type, a: float, b: float) -> float:
    """A simple calculator tool that can add, multiply, substract, or divide two numbers."""
    if operator == "add":
        return a + b
    elif operator == "multiply":
        return a * b
    elif operator == "substraction":
        return a - b
    elif operator == "division":
        return a / b
    else:
        raise ToolException("Invalid operator")

In [33]:
print(calculator.name)
print(calculator.description)
print(calculator.return_direct)
print(calculator.args)

Calculator-tool
Calculator-tool(operator: Literal['add', 'multiply', 'substraction', 'division'], a: float, b: float) -> float - A simple calculator tool that can add, multiply, substract, or divide two numbers.
False
{'operator': {'title': 'Operator', 'enum': ['add', 'multiply', 'substraction', 'division'], 'type': 'string'}, 'a': {'title': 'A', 'type': 'number'}, 'b': {'title': 'B', 'type': 'number'}}


In [34]:
calculator.run({"operator": "add", "a": 2, "b": 3})

5.0

### Building A Tool Calling Agent

In [35]:
tools = [calculator]

In [36]:
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [37]:
agent = create_openai_tools_agent(llm, tools, prompt)

In [38]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [39]:
response = agent_executor.invoke({
    "input": "What is the product of 2 and 3?",
})



> Entering new AgentExecutor chain...

Invoking: `Calculator-tool` with `{'operator': 'multiply', 'a': 2, 'b': 3}`


6.0The product of 2 and 3 is 6.

> Finished chain.


## Natural Language To Cypher

In [40]:
import os
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.prompts import (
    PromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
    ChatPromptTemplate
)

In [51]:
neo4j_graph_vector_index = Neo4jVector.from_existing_graph(
    embedding=OpenAIEmbeddings(api_key=os.getenv("OPENAI_API_KEY")),
    url=os.getenv("NEO4J_URL"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
    index_name="employee",
    node_label="Employee",
    text_node_properties=[
        "address",
        "city",
        "title",
        "email",
        "postalCode"
        "note",
        "firstName",
        "lastName",
        "region"
    ],
    embedding_node_property="embedding",
)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated procedure. ('db.create.setVectorProperty' has been replaced by 'db.create.setNodeVectorProperty')} {position: line: 1, column: 70, offset: 69} for query: "UNWIND $data AS row MATCH (n:`Employee`) WHERE elementId(n) = row.id CALL db.create.setVectorProperty(n, 'embedding', row.embedding) YIELD node RETURN count(*)"


In [52]:
result = neo4j_graph_vector_index.similarity_search("Andrew", k=2)

In [53]:
result

[Document(page_content='\naddress: 507 20th Ave. E. Apt. 2A\ncity: Seattle\ntitle: Sales Representative\nemail: \npostalCodenote: \nfirstName: Nancy\nlastName: Davolio\nregion: WA', metadata={'employeeID': 1, 'hireDate': '1992-05-01 00:00:00.000', 'photo': '0x151C2F00020000000D000E0014002100FFFFFFFF4269746D617020496D616765005061696E742E506963747572650001050000020000000700000050427275736800000000000000000020540000424D20540000000000007600000028000000C0000000DF0000000100040000000000A0530000CE0E0000D80E0000000000', 'homePhone': '(206) 555-9857', 'photoPath': 'http://accweb/emmployees/davolio.bmp', 'country': 'USA', 'extension': 5467, 'postalCode': '98122', 'titleOfCourtesy': 'Ms.', 'birthDate': '1948-12-08 00:00:00.000', 'notes': 'Education includes a BA in psychology from Colorado State University in 1970.  She also completed The Art of the Cold Call.  Nancy is a member of Toastmasters International.'}),
 Document(page_content='\naddress: 14 Garrett Hill\ncity: London\ntitle: Sales Manage

In [54]:
print(result[0].metadata)

{'employeeID': 1, 'hireDate': '1992-05-01 00:00:00.000', 'photo': '0x151C2F00020000000D000E0014002100FFFFFFFF4269746D617020496D616765005061696E742E506963747572650001050000020000000700000050427275736800000000000000000020540000424D20540000000000007600000028000000C0000000DF0000000100040000000000A0530000CE0E0000D80E0000000000', 'homePhone': '(206) 555-9857', 'photoPath': 'http://accweb/emmployees/davolio.bmp', 'country': 'USA', 'extension': 5467, 'postalCode': '98122', 'titleOfCourtesy': 'Ms.', 'birthDate': '1948-12-08 00:00:00.000', 'notes': 'Education includes a BA in psychology from Colorado State University in 1970.  She also completed The Art of the Cold Call.  Nancy is a member of Toastmasters International.'}


In [55]:
result = neo4j_graph_vector_index.similarity_search(
    "Employee detail", filter={"country": "USA"}
)

In [56]:
result

[Document(page_content='\naddress: 507 20th Ave. E. Apt. 2A\ncity: Seattle\ntitle: Sales Representative\nemail: \npostalCodenote: \nfirstName: Nancy\nlastName: Davolio\nregion: WA', metadata={'employeeID': 1, 'hireDate': '1992-05-01 00:00:00.000', 'photo': '0x151C2F00020000000D000E0014002100FFFFFFFF4269746D617020496D616765005061696E742E506963747572650001050000020000000700000050427275736800000000000000000020540000424D20540000000000007600000028000000C0000000DF0000000100040000000000A0530000CE0E0000D80E0000000000', 'homePhone': '(206) 555-9857', 'photoPath': 'http://accweb/emmployees/davolio.bmp', 'country': 'USA', 'extension': 5467, 'postalCode': '98122', 'titleOfCourtesy': 'Ms.', 'birthDate': '1948-12-08 00:00:00.000', 'notes': 'Education includes a BA in psychology from Colorado State University in 1970.  She also completed The Art of the Cold Call.  Nancy is a member of Toastmasters International.'}),
 Document(page_content='\naddress: 722 Moss Bay Blvd.\ncity: Kirkland\ntitle: Sales R

In [57]:
for employee in result:
    clearned_page_content = dict(line.split(": ", 1) for line in employee.page_content.strip().split("\n"))
    
    print(f"""
          First name: {clearned_page_content.get('firstName')}
          Last name: {clearned_page_content.get('lastName')}
          Title: {clearned_page_content.get('title')}
          country: {employee.metadata.get('country')}
          """)


          First name: Nancy
          Last name: Davolio
          Title: Sales Representative
          country: USA
          

          First name: Janet
          Last name: Leverling
          Title: Sales Representative
          country: USA
          

          First name: Anne
          Last name: Dodsworth
          Title: Sales Representative
          country: UK
          

          First name: Steven
          Last name: Buchanan
          Title: Sales Manager
          country: UK
          


In [58]:
result = neo4j_graph_vector_index.similarity_search(
    "Employee detail", filter={"country": {"$ne": 'UK'}}, k=2
)

In [59]:
result

[Document(page_content='\naddress: 507 20th Ave. E. Apt. 2A\ncity: Seattle\ntitle: Sales Representative\nemail: \npostalCodenote: \nfirstName: Nancy\nlastName: Davolio\nregion: WA', metadata={'employeeID': 1, 'hireDate': '1992-05-01 00:00:00.000', 'photo': '0x151C2F00020000000D000E0014002100FFFFFFFF4269746D617020496D616765005061696E742E506963747572650001050000020000000700000050427275736800000000000000000020540000424D20540000000000007600000028000000C0000000DF0000000100040000000000A0530000CE0E0000D80E0000000000', 'homePhone': '(206) 555-9857', 'photoPath': 'http://accweb/emmployees/davolio.bmp', 'country': 'USA', 'extension': 5467, 'postalCode': '98122', 'titleOfCourtesy': 'Ms.', 'birthDate': '1948-12-08 00:00:00.000', 'notes': 'Education includes a BA in psychology from Colorado State University in 1970.  She also completed The Art of the Cold Call.  Nancy is a member of Toastmasters International.'}),
 Document(page_content='\naddress: 722 Moss Bay Blvd.\ncity: Kirkland\ntitle: Sales R

$ne
$lt
$lte
$gte
$gt
$in
$like
$ilke

In [60]:
for employee in result:
    clearned_page_content = dict(line.split(": ", 1) for line in employee.page_content.strip().split("\n"))
    
    print(f"""
          First name: {clearned_page_content.get('firstName')}
          Last name: {clearned_page_content.get('lastName')}
          Title: {clearned_page_content.get('title')}
          country: {employee.metadata.get('country')}
          """)


          First name: Nancy
          Last name: Davolio
          Title: Sales Representative
          country: USA
          

          First name: Janet
          Last name: Leverling
          Title: Sales Representative
          country: USA
          


In [61]:
employee_details_chat_template_str = """
Your job is to use the provided employee data to answer 
questions about their roles, performance, and experiences 
within the company. Use the following context to answer questions. 
Be as detailed as possible, but don't make up any information that's 
not from the context. If you don't know an answer, say you don't know.

Context: {context}
"""

In [62]:
employee_details_chat_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"],
        template=employee_details_chat_template_str
    )
)

In [63]:
human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["question"],
        template="Can you provide details on: {question}?"
    )
)

In [64]:
messages = [employee_details_chat_system_prompt, human_prompt]

In [65]:
qa_prompt = ChatPromptTemplate(
    messages=messages,
    input_variables=["context", "question"]
)

In [66]:
llm = ChatOpenAI(model="gpt-3.5-turbo")

In [67]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=neo4j_graph_vector_index.as_retriever(),
    chain_type="stuff",
)

In [68]:
qa_chain.combine_documents_chain.llm_chain.prompt = qa_prompt

In [69]:
response = qa_chain.invoke("Give me profile details of Dodsworth.")

In [70]:
print(response.get("result"))

Anne Dodsworth is a Sales Representative at the company, located at 7 Houndstooth Rd. in London. She is part of the sales team and is responsible for handling sales activities for the company. Anne's email address and postal code are not provided in the context. Her region is listed as "Unknown." If you need more specific information about Anne Dodsworth, please let me know.


In [71]:
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain

In [72]:
graph = Neo4jGraph(
    url=os.getenv("NEO4J_URL"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
)

In [73]:
graph.refresh_schema()

In [74]:
cypher_generation_template = """
Task:
Generate Cypher query for a Neo4j graph database.

Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.

Schema:
{schema}

Note:
Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything other than
for you to construct a Cypher statement. Do not include any text except
the generated Cypher statement. Make sure the direction of the relationship is
correct in your queries. Make sure you alias both entities and relationships
properly. Do not run any queries that would add to or delete from
the database. Make sure to alias all statements that follow as with
statement (e.g. WITH c as customer, o.orderID as order_id).
If you need to divide numbers, make sure to
filter the denominator to be non-zero.

Examples:
# Retrieve the total number of orders placed by each customer.
MATCH (c:Customer)-[o:ORDERED_BY]->(order:Order)
RETURN c.customerID AS customer_id, COUNT(o) AS total_orders
# List the top 5 products with the highest unit price.
MATCH (p:Product)
RETURN p.productName AS product_name, p.unitPrice AS unit_price
ORDER BY unit_price DESC
LIMIT 5
# Find all employees who have processed orders.
MATCH (e:Employee)-[r:PROCESSED_BY]->(o:Order)
RETURN e.employeeID AS employee_id, COUNT(o) AS orders_processed
String category values:
Use existing strings and values from the schema provided. 

The question is:
{question}
"""

In [75]:
cypher_generation_prompt = PromptTemplate(
    input_variables=["schema", "question"],
    template=cypher_generation_template
)

In [76]:
qa_generation_template_str = """
You are an assistant that takes the results from a Neo4j Cypher query and forms a human-readable response. The query results section contains the results of a Cypher query that was generated based on a user's natural language question. The provided information is authoritative; you must never question it or use your internal knowledge to alter it. Make the answer sound like a response to the question.

Query Results:
{context}
Question:
{question}
If the provided information is empty, respond by stating that you don't know the answer. Empty information is indicated by: []
If the information is not empty, you must provide an answer using the results. If the question involves a time duration, assume the query results are in units of days unless specified otherwise.
When names are provided in the query results, such as hospital names, be cautious of any names containing commas or other punctuation. For example, 'Jones, Brown and Murray' is a single hospital name, not multiple hospitals. Ensure that any list of names is presented clearly to avoid ambiguity and make the full names easily identifiable.
Never state that you lack sufficient information if data is present in the query results. Always utilize the data provided.
Helpful Answer:
"""

In [77]:
qa_generation_prompt = PromptTemplate(
    input_variables=["context", "question"], template=qa_generation_template_str
)

In [78]:
cypher_chain = GraphCypherQAChain.from_llm(
    top_k=10,
    graph=graph,
    verbose=True,
    validate_cypher=True,
    qa_prompt=qa_generation_prompt,
    cypher_prompt=cypher_generation_prompt,
    qa_llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0),
    cypher_llm=ChatOpenAI(model="gpt-4o-mini", temperature=0),
)

In [79]:
question = "What is the most expensive product?"
response = cypher_chain.invoke(question)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Product)
RETURN p.productName AS product_name, p.unitPrice AS unit_price
ORDER BY unit_price DESC
LIMIT 1
Full Context:
[{'product_name': 'Côte de Blaye', 'unit_price': 263.5}]

> Finished chain.


In [80]:
print(response.get("result"))

The most expensive product is Côte de Blaye, priced at $263.50.


In [81]:
question = "List the top 5 products with the highest unit price."
response = cypher_chain.invoke(question)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Product)
RETURN p.productName AS product_name, p.unitPrice AS unit_price
ORDER BY unit_price DESC
LIMIT 5

Full Context:
[{'product_name': 'Côte de Blaye', 'unit_price': 263.5}, {'product_name': 'Thüringer Rostbratwurst', 'unit_price': 123.79}, {'product_name': 'Mishi Kobe Niku', 'unit_price': 97.0}, {'product_name': "Sir Rodney's Marmalade", 'unit_price': 81.0}, {'product_name': 'Carnarvon Tigers', 'unit_price': 62.5}]

> Finished chain.


In [82]:
print(response.get("result"))

The top 5 products with the highest unit prices are:
1. Côte de Blaye - $263.5
2. Thüringer Rostbratwurst - $123.79
3. Mishi Kobe Niku - $97.0
4. Sir Rodney's Marmalade - $81.0
5. Carnarvon Tigers - $62.5


In [83]:
question = "Give me more details on each shipper as well as how many orders they have shipped."
response = cypher_chain.invoke(question)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (s:Shipper)<-[sh:SHIPPED_BY]-(o:Order)
RETURN s.shipperID AS shipper_id, s.companyName AS company_name, s.phone AS phone, COUNT(o) AS total_orders_shipped

Full Context:
[{'shipper_id': 3, 'company_name': 'Federal Shipping', 'phone': '(503) 555-9931', 'total_orders_shipped': 87}, {'shipper_id': 1, 'company_name': 'Speedy Express', 'phone': '(503) 555-9831', 'total_orders_shipped': 67}, {'shipper_id': 2, 'company_name': 'United Package', 'phone': '(503) 555-3199', 'total_orders_shipped': 96}]

> Finished chain.


In [84]:
print(response.get("result"))

Sure! Here are the details on each shipper along with the total number of orders they have shipped:

1. Shipper ID: 3
   Company Name: Federal Shipping
   Phone: (503) 555-9931
   Total Orders Shipped: 87

2. Shipper ID: 1
   Company Name: Speedy Express
   Phone: (503) 555-9831
   Total Orders Shipped: 67

3. Shipper ID: 2
   Company Name: United Package
   Phone: (503) 555-3199
   Total Orders Shipped: 96

These are the details you requested!


## Agents In GraphRAG Systems

In [85]:
employee_details_chat_template_str = """
Your job is to use the provided employee data to answer 
questions about their roles, performance, and experiences 
within the company. Use the following context to answer questions. 
Be as detailed as possible, but don't make up any information that's 
not from the context. If you don't know an answer, say you don't know.
{context}
"""

In [86]:
@tool("employee-qa-tool", return_direct=False)
def employee_qa_tool(query: str) -> str:
    """Useful for answering questions about employees who work at a company."""
    
    employee_details_chat_system_prompt = SystemMessagePromptTemplate(
        prompt=PromptTemplate(
            input_variables=["context"],
            template=employee_details_chat_template_str
        )
    )
    
    human_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            input_variables=["question"],
            template="Can you provide details on: {question}?"
        )
    )
    
    messages = [employee_details_chat_system_prompt, human_prompt]
    
    qa_prompt = ChatPromptTemplate(
        messages=messages,
        input_variables=["context", "question"]
    )
    
    llm = ChatOpenAI(model="gpt-3.5-turbo")
    
    
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=neo4j_graph_vector_index.as_retriever(),
        # ['stuff', 'map_reduce', 'refine', 'map_rerank']
        chain_type="stuff",
    )
    
    qa_chain.combine_documents_chain.llm_chain.prompt = qa_prompt
    
    response = qa_chain.invoke(query)
    
    return response.get("result")

In [89]:
employee_qa_tool.invoke("What is the address of the Vice president Andrew?")

'The address of the Vice President, Andrew Fuller, is 908 W. Capital Way, Tacoma, WA.'

In [90]:
@tool("general-qa-tool", return_direct=False)
def general_qa_tool(query: str) -> str:
    """Useful for answering general questions about orders, order details, shipper, customers, and products."""
    response = cypher_chain.invoke(query)
    
    return response.get("result")

In [91]:
general_qa_tool.invoke("What is the most expensive product?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Product)
RETURN p.productName AS product_name, p.unitPrice AS unit_price
ORDER BY unit_price DESC
LIMIT 1
Full Context:
[{'product_name': 'Côte de Blaye', 'unit_price': 263.5}]

> Finished chain.


'The most expensive product is "Côte de Blaye" with a unit price of 263.5.'

### Creating Agent

In [92]:
from langchain import hub
from langchain.agents import (
    AgentExecutor,
    create_openai_tools_agent
)

In [93]:
agent_prompt = hub.pull("hwchase17/openai-functions-agent")

In [94]:
agent_prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [95]:
tools = [employee_qa_tool, general_qa_tool]

In [96]:
llm = ChatOpenAI(model="gpt-3.5-turbo")

In [97]:
agent = create_openai_tools_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [98]:
agent_executor.invoke({
    "input": "What is the most expensive product?"
})



> Entering new AgentExecutor chain...

Invoking: `general-qa-tool` with `{'query': 'most expensive product'}`




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Product)
RETURN p.productName AS product_name, p.unitPrice AS unit_price
ORDER BY unit_price DESC
LIMIT 1
Full Context:
[{'product_name': 'Côte de Blaye', 'unit_price': 263.5}]

> Finished chain.
The most expensive product is "Côte de Blaye" with a unit price of 263.5.The most expensive product is "Côte de Blaye" with a unit price of $263.5.

> Finished chain.


{'input': 'What is the most expensive product?',
 'output': 'The most expensive product is "Côte de Blaye" with a unit price of $263.5.'}

In [102]:
agent_executor.invoke({
    "input": "How many products were ordered by customers, of which the shipper shipped them the earliest and which Employees processed them?"
})



> Entering new AgentExecutor chain...

Invoking: `general-qa-tool` with `{'query': 'number of products ordered by customers'}`




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Customer)<-[o:ORDERED_BY]-(order:Order)-[i:INCLUDES]->(p:Product)
RETURN c.customerID AS customer_id, COUNT(p) AS total_products_ordered
Full Context:
[{'customer_id': 'VINET', 'total_products_ordered': 6}, {'customer_id': 'TOMSP', 'total_products_ordered': 2}, {'customer_id': 'HANAR', 'total_products_ordered': 6}, {'customer_id': 'VICTE', 'total_products_ordered': 5}, {'customer_id': 'SUPRD', 'total_products_ordered': 6}, {'customer_id': 'CHOPS', 'total_products_ordered': 3}, {'customer_id': 'RICSU', 'total_products_ordered': 4}, {'customer_id': 'WELLI', 'total_products_ordered': 2}, {'customer_id': 'HILAA', 'total_products_ordered': 3}, {'customer_id': 'ERNSH', 'total_products_ordered': 7}]

> Finished chain.
The number of products ordered by customers are as follows:
- VINET: 6 produ

{'input': 'How many products were ordered by customers, of which the shipper shipped them the earliest and which Employees processed them?',
 'output': 'The products were ordered by customers in the quantities mentioned earlier. The shipper that shipped them the earliest is Speedy Express, with an earliest shipped date of July 10, 1996. The employees who processed these orders are Nancy Davolio and Robert King.'}